In [1]:
# Python 샘플 코드 #
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [15]:
# 해당 코드에서 크롤링을 시도할 연도(yyyy00)
this_year = 201600

In [2]:
'''
국토교통부_아파트매매 실거래 상세 자료
활용기간 2023.04.12 까지
LAWD_CD = 11110  # 47290(경산) 지역코드 - 각 지역별 코드 행정표준코드관리시스템(www.code.go.kr)의 법정동코드 10자리 중 앞 5자리
DEAL_YMD = 200601  # 계약월 - 실거래 자료의 계약년월 6자리
numOfRows = 500  # 한 페이지 결과 수
pageNo = 1  # 페이지번호
'''
def get_apt_data(LAWD_CD, DEAL_YMD, serviceKey, numOfRows=10000):
    # 호출하려는 OpenAPI URL 정의
    data_url = "".join(
        ["http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?",
         "serviceKey=", serviceKey, "&LAWD_CD=", str(LAWD_CD), "&DEAL_YMD=", str(DEAL_YMD), "&numOfRows=",
         str(numOfRows)])

    # 정의된 OpenAPI URL을 호출
    data_response = requests.get(data_url)
    data_xml = BeautifulSoup(data_response.text, "xml")
    data_xml_row = data_xml.find_all("item")
    data_xml_row

    # XML에 포함된 항목들을 리스트로 저장
    list_items = ["거래금액", "건축년도", "년", "도로명", "도로명건물본번호코드", "도로명건물부번호코드",
                  "도로명시군구코드", "도로명일련번호코드", "도로명지상지하코드", "도로명코드",
                  "법정동", "법정동본번코드", "법정동부번코드", "법정동시군구코드", "법정동읍면동코드", "법정동지번코드",
                  "아파트", "월", "일", "일련번호", "전용면적", "지번", "지역코드", "층", "해제사유발생일", "해제여부"]

    # 리스트의 리스트 형태로 데이터프레임에 저장할 자료 만들기
    list_dataframe = []

    for n_obs in range(0, len(data_xml_row)):
        list_obs_values = []  # 동일한 관측치의 데이터는 모두 이 리스트에 저장된다
        for i in range(0, len(list_items)):
            # 특정 항목이 XML에 아예 존재하지 않는 경우가 있으므로 try 구문 사용
            try:
                list_obs_values.append(data_xml_row[n_obs].find(list_items[i]).text)
            except Exception as e:
                list_obs_values.append("")
                print("Error in " + str(n_obs) + "th obs, item: " + list_items[i] + ", " + str(e))
        list_dataframe.append(list_obs_values)  # 리스트의 리스트로 관측치 저장

    # 데이터프레임 형태로 변환
    df_apt = pd.DataFrame(list_dataframe, columns=list_items)
    return df_apt

In [3]:
# 함수 작동 여부 확인
serviceKey_HKim = "L2bxTcSSMqCj6TrXC3gwMUrZ6N34nwOC9RkkqQiW0ZlxY2MhEaeqpgEcyj8nNJ%2F8UO8tv76Q2YVLo5e01tL15A%3D%3D"
serviceKey_YKim = "%2F2a7hudNIqdsS1Uq7SaicUQYvpno8zZ4M53iUr4PgZnXIDq4SMhbDLNHrk6PfFXNdnb7LuNpEffhq8YXRq2dbQ%3D%3D"
serviceKey_ELee = "qsGZz9dk4cJcJQl49SOrzReTZlntOB%2FxGF1aXfwPUeyr1HOhvWaKySoBN1sqiKX3gPavncLjqKGAAkI4ErVsGg%3D%3D"
serviceKey_JKim = "%2B8oYQ2dI4wTSizbFCnqyo3ssuRB%2Fl%2BIrzomjcEwJq4csP9z%2F%2ByjUJZkqjOiz8HOXQyGII6edaXCxIq7v5%2FvUhw%3D%3D"
serviceKey_Chun = "2ix6etGjWbVvOzBAIKHx7H6/APHUZPbYUJM/UiVE8kqSUPIEEJp+hH9RBil1kfrFYeQSfJzfLDKR50JL0TKaGg=="
df_tmp1 = get_apt_data(LAWD_CD=11110, DEAL_YMD=202102, serviceKey=serviceKey_Chun)


Error in 17th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 18th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 19th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 19th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 20th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 20th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 21th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 21th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 25th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 31th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 36th obs, item: 일련번호, 'NoneType' object has no attribute 'text'


In [5]:
# Import TXT file 법정동코드 전체자료 불러오기
df_lawd_cd = pd.read_csv('C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/법정동코드 전체자료.txt', sep="\t", engine='python', encoding="CP949")
df_lawd_cd["LAWD_CD"] = df_lawd_cd["법정동코드"].divmod(100000)[0]
df_lawd_cd_nodup = df_lawd_cd.drop_duplicates(subset=['LAWD_CD']).copy()  # 중복값 제외
df_lawd_cd_nodup_exist = df_lawd_cd_nodup[df_lawd_cd_nodup["폐지여부"] == "존재"].copy()

In [6]:
total_lawd_cd = df_lawd_cd_nodup_exist["LAWD_CD"]
tmp_lawd_cd = df_lawd_cd_nodup_exist["LAWD_CD"][3:10]

In [7]:
# 국토교통부_아파트매매 실거래 상세 자료는 2008년 1월부터 자료가 존재한다.
# 그런데 2006년으로 입력해도 자료가 있다.

In [8]:

# 시작 시점과 종료 시점 입력
yyyymm_start = this_year + 7  # 시작 년월
yyyymm_end = this_year + 9  # 종료 년월

list_yyyymm = []
for n_yyyymm in range(yyyymm_start, yyyymm_end+1):
    if (divmod(n_yyyymm, 100)[1] != 0) & (divmod(n_yyyymm, 100)[1] < 13):
        list_yyyymm.append(n_yyyymm)

list_yyyymm.reverse()


# StopWatch: 코드 시작
time_this_code_start = datetime.now()
print("This code started at: " + str(time_this_code_start))

# DataFrame 구성을 위해 공백 자료를 하나 만든다.
df_dataset_null = get_apt_data(LAWD_CD=11120, DEAL_YMD=202102, serviceKey=serviceKey_Chun)

# 전국의 한달 자료 받는데 약 9분 소요
for yyyymm in list_yyyymm:
    df_dataset = df_dataset_null.copy()
    for lawd_cd in total_lawd_cd:
        df_apt = get_apt_data(LAWD_CD=lawd_cd, DEAL_YMD=yyyymm, serviceKey=serviceKey_Chun)
        # df_apt.to_pickle("./data_raw/df_apt_" + str(yyyymm) + "_" + str(lawd_cd) + ".pkl")
        df_dataset = pd.concat([df_dataset, df_apt])
    df_dataset.to_pickle("C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_" + str(yyyymm) + ".pkl")

# StopWatch: 코드 종료
time_this_code_end = datetime.now()
print("This code finished at: " + str(time_this_code_end))
print("Elapsed (in this code): " + str(time_this_code_end - time_this_code_start))

This code started at: 2021-05-09 02:28:11.399320
Error in 45th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 112th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 123th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 124th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 124th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 124th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in 124th obs, item: 도로명시군구코드, 'NoneType' object has no attribute 'text'
Error in 124th obs, item: 도로명코드, 'NoneType' object has no attribute 'text'
Error in 124th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 67th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 72th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 73th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 79th obs, item: 도로명지상지

Error in 11th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 15th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 29th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 33th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 45th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 48th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 54th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 56th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 58th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 59th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 62th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 72th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 74th obs, item: 도로명지상지하코드, 'NoneType' object has no att

Error in 11th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 28th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 37th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 38th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 40th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 41th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 46th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 51th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 54th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 89th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 94th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 95th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 102th obs, item: 도로명지상지하코드, 'NoneType' object has no at

Error in 268th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 275th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 284th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 288th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 294th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 300th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 307th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 320th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 331th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 351th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 356th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 359th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 363th obs, item: 도로명지상지하코드, 'NoneType' object ha

Error in 15th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 16th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 76th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 101th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 118th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 119th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 120th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 121th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 122th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 123th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 124th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 125th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 126th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 127th obs, item: 일련번호,

...

Error in 242th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 343th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 360th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 364th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 6th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 16th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 19th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 22th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 37th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 40th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 43th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 310th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 325th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 327th obs, i

Error in 115th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 116th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 26th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 26th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 28th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 30th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 31th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 80th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 117th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 118th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 176th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 179th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 184th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'tex

Error in 173th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 175th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 178th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 181th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 184th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 186th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 188th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 189th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 195th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 197th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 198th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 200th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 213th obs, item: 도로명지상지하코드, 'NoneType' obje

Error in 464th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 481th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 482th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 483th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 484th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 485th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 0th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 26th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 33th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 34th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 35th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 36th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 37th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 38th ob

Error in 1th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 3th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 5th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 230th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 238th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 3th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 4th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 5th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 55th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 57th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 67th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 68th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 69th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Er

In [11]:
# Check the data

data1 = pd.read_pickle('C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_'+str(yyyymm_start)+'.pkl')

data2 = pd.read_pickle('C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_'+str(yyyymm_start + 1)+'.pkl')

data3 = pd.read_pickle('C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_'+str(yyyymm_start + 2)+'.pkl')

In [12]:
data1

,거래금액,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,도로명코드,...,아파트,월,일,일련번호,전용면적,지번,지역코드,층,해제사유발생일,해제여부


In [13]:
data2

,거래금액,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,도로명코드,...,아파트,월,일,일련번호,전용면적,지번,지역코드,층,해제사유발생일,해제여부
0,"91,000",2000,2017,자하문로33길,00043,00000,11110,01,0,4100285,...,청운현대,12,19,11110-4,129.76,56-45,11110,5,,
1,"54,000",1973,2017,사직로,00095,00000,11110,02,0,3100005,...,사직파크맨션,12,10,11110-211,99.17,142,11110,5,,
2,"140,000",2008,2017,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),12,2,11110-2203,158.99,9,11110,12,,
3,"89,900",2008,2017,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),12,7,11110-2203,121.37,9,11110,2,,
4,"111,000",2008,2017,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),12,8,11110-2203,158.99,9,11110,1,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,"8,200",1996,2017,상진2로,00017,00000,43800,01,0,3248032,...,두진임대아파트,12,11,43800-10,59.34,50,43800,4,,
6,"4,800",1996,2017,상진2로,00017,00000,43800,01,0,3248032,...,두진임대아파트,12,12,43800-10,35.67,50,43800,5,,
7,"6,950",1996,2017,상진2로,00017,00000,43800,01,0,3248032,...,두진임대아파트,12,13,43800-10,49.69,50,43800,5,,
8,"7,350",1996,2017,상진2로,00017,00000,43800,01,0,3248032,...,두진임대아파트,12,20,43800-10,49.69,50,43800,3,,


In [14]:
data3

,거래금액,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,도로명코드,...,아파트,월,일,일련번호,전용면적,지번,지역코드,층,해제사유발생일,해제여부
0,"91,000",2000,2017,자하문로33길,00043,00000,11110,01,0,4100285,...,청운현대,12,19,11110-4,129.76,56-45,11110,5,,
1,"54,000",1973,2017,사직로,00095,00000,11110,02,0,3100005,...,사직파크맨션,12,10,11110-211,99.17,142,11110,5,,
2,"140,000",2008,2017,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),12,2,11110-2203,158.99,9,11110,12,,
3,"89,900",2008,2017,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),12,7,11110-2203,121.37,9,11110,2,,
4,"111,000",2008,2017,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),12,8,11110-2203,158.99,9,11110,1,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,"8,200",1996,2017,상진2로,00017,00000,43800,01,0,3248032,...,두진임대아파트,12,11,43800-10,59.34,50,43800,4,,
6,"4,800",1996,2017,상진2로,00017,00000,43800,01,0,3248032,...,두진임대아파트,12,12,43800-10,35.67,50,43800,5,,
7,"6,950",1996,2017,상진2로,00017,00000,43800,01,0,3248032,...,두진임대아파트,12,13,43800-10,49.69,50,43800,5,,
8,"7,350",1996,2017,상진2로,00017,00000,43800,01,0,3248032,...,두진임대아파트,12,20,43800-10,49.69,50,43800,3,,
